### Preparation to build a code for ANN training

*   Access to your google drive
*   Import tensorflow framework to run deep learning model
*   Allocate GPU

In [ ]:
# Access to Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Import 'Tensorflow' package
import tensorflow as tf
from tensorflow import keras

# Check the version of tensorflow
print(tf.__version__)

In [ ]:
# Check if a GPU(in Google server) is allocated
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')

print('Found GPU at: {}'.format(device_name))

## Step 5-1: ANN
We've extracted the features of the classified data in ML8_1.

We'll train ANN model using those feature data.

Plase refer ML 7_1 for ANN grid search.

Prepare the selected features data

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Load selected feature data and P-values at Step 4-1 in ML8_1
feature_path = "YOUR_PATH/FeatureSelected.csv"
p_rank_path = 'YOUR_PATH/P_value_Rank.csv'

FeatureSelected = pd.read_csv(feature_path, header=None)
P_value_Rank = pd.read_csv(p_rank_path , header=None)

# Standardize the selected features
FeatureSelected_std = StandardScaler().fit_transform(pd.DataFrame(FeatureSelected).T)
FeatureSelected_std.shape

(623, 30)

In [ ]:
# Split feature data into 6 classes back
num_class=6
num_data=[96, 161, 144, 41, 34, 147] # 1 segment per 1 second with 0.8 second overlap


# Index feature data



Split dataset into training and test sets

In [ ]:
from sklearn.model_selection import train_test_split

# Define the test data ratio
test_ratio = 0.2

# Split the datasets into training and test sets

### You need to split all classified data into train and test sets.
### Examples below.
TRAIN_DATA_CLASS1, TEST_DATA_CLASS1 = train_test_split(DATA_CLASS1, test_size=test_ratio, random_state=777)
TRAIN_DATA_CLASS2, TEST_DATA_CLASS2 = train_test_split(DATA_CLASS2, test_size=test_ratio, random_state=777)
...



In [ ]:
### How to handle iterative codes
### This is how to make your code more organized; it's not an essential thing!

### If you need to perform the same task repeatedly and you're doing it one by one,
### that would be like below.
TRAIN_DATA_CLASS1, TEST_DATA_CLASS1 = train_test_split(DATA_CLASS1, test_size=test_ratio, random_state=777)
TRAIN_DATA_CLASS2, TEST_DATA_CLASS2 = train_test_split(DATA_CLASS2, test_size=test_ratio, random_state=777)
TRAIN_DATA_CLASS3, TEST_DATA_CLASS3 = train_test_split(DATA_CLASS3, test_size=test_ratio, random_state=777)
TRAIN_DATA_CLASS4, TEST_DATA_CLASS4 = train_test_split(DATA_CLASS4, test_size=test_ratio, random_state=777)
TRAIN_DATA_CLASS5, TEST_DATA_CLASS5 = train_test_split(DATA_CLASS5, test_size=test_ratio, random_state=777)
TRAIN_DATA_CLASS6, TEST_DATA_CLASS6 = train_test_split(DATA_CLASS6, test_size=test_ratio, random_state=777)

### You can make these repeated codes as a 'for' loop
for i in range(num_class): # i starts from '0' and ends with 'num_class-0'
  exec(f"TRAIN_DATA_CLASS{i+1}, TEST_DATA_CLASS{i+1} = train_test_split(DATA_CLASS{i+1}, test_size=test_ratio, random_state=777)")

In [ ]:
# Make label (One-hot Encoding)
### You need to make label for training and test.
### For class 1, the label is like [1 0 0 0 0 0],
### for class 2, the label is like [0 1 0 0 0 0], and so on ...
### Examples below (You can use a loop of course!)
TRAIN_LABEL_CLASS1=np.zeros((TRAIN_DATA_CLASS1.shape[0],num_class))
TRAIN_LABEL_CLASS1[:,0]=1
TEST_LABEL_CLASS1=np.zeros((TEST_DATA_CLASS1.shape[0],num_class))
TEST_LABEL_CLASS1[:,0]=1
...



# Combine the classified data/labels
### Now, you need to combine all classified train and test data into train and test dataset
### and train and test label into a train label set and test label set.
TrainData = np.concatenate([TRAIN_DATA_CLASS1, TRAIN_DATA_CLASS2, ..., TRAIN_DATA_CLASS6], axis=0)
TestData = np.concatenate([TEST_DATA_CLASS1, TEST_DATA_CLASS2, ..., TEST_DATA_CLASS6], axis=0)
TrainLabel = np.concatenate([TRAIN_LABEL_CLASS1, TRAIN_LABEL_CLASS2, ..., TRAIN_LABEL_CLASS6], axis=0)
TestLabel = np.concatenate([TEST_LABEL_CLASS1, TEST_LABEL_CLASS2, ..., TEST_LABEL_CLASS6], axis=0)

print("- total train data", TrainData.shape,  ", test data", TestData.shape)
print("- total train label", TrainLabel.shape, ", test label",TestLabel.shape)

Prepare lists of hyperparameters for grid search

In [ ]:
# Set hyperparameters for grid search
activation_function = ['tanh', 'relu'] # activation function
hidden_layer = [2, 3] # number of hiddent layers
learning_rate = [0.001, 0.01] # learning rate

# Set hyperparameters for each training
num_neuron = 16
epoch = 300

# Calculate the number of cases
num_case = len(activation_function) * len(hidden_layer) * len(learning_rate)
print("We're gonna run", num_case, "models")

In [ ]:
# Define a function to create ANN models by inputting the hyperparameters for grid search
### You can refer ML7_1
### Please keep in mind that the output node units should be same with the number of classes
def ANN_model(input_data, num_neuron, temp_actfn, temp_layer, temp_lr):





  return model

In [ ]:
# Create an empty dataframe to store the accuracy results
Accuracy_df = pd.DataFrame(np.zeros(shape=(num_case, 4)), columns=['Activation Function', 'Num of hidden layer', 'Learning rate', 'Accuracy'])

Train the ANN models with different combinations of hyperparameters and save them

In [ ]:
# Initialize a count value to store the performance of each model
cnt = 0

# Iterate through all possible combinations of activation functions, hidden layers, and learning rates
for act_func in activation_function: # Select each activation function in the list
    for layer in hidden_layer: # Select each hidden layer configuration in the list
        for lr in learning_rate: # Select each learning rate value in the list

            # Create, train, and validate a temporary ANN model with the current combination of hyperparameters
            temp_ann_model = ...


            # Save the temporary model to a file with a corresponding name
            temp_ann_model_name = f'ANN_{act_func}_L{layer}_LR{lr:.4f}.h5'
            temp_ann_model.save('YOUR_PATH/GridSearch_ANN/' + temp_ann_model_name)

            # Store the performance (accuracy) of the temporary model in the dataframe
            Accuracy_df.iloc[cnt, :] = [act_func, layer, lr, Accuracy]
            cnt += 1

# Display the resulting dataframe with model performances
Accuracy_df

Confirm the grid search results

In [ ]:
# Calculate mean and standard deviation accuracy for each activation function
mean_accuracy_actfn = Accuracy_df.groupby(['Activation Function'])['Accuracy'].agg(['mean', 'std']).reset_index()

# Calculate mean and standard deviation of accuracy for each hidden layer
mean_accuracy_layer = Accuracy_df.groupby(['Num of hidden layer'])['Accuracy'].agg(['mean', 'std']).reset_index()

# Calculate mean and standard deviation of accuracy for each learning rate
mean_accuracy_lr = Accuracy_df.groupby(['Learning rate'])['Accuracy'].agg(['mean', 'std']).reset_index()

print("Mean accuarcy\n", mean_accuracy_actfn,"\n\n", mean_accuracy_layer, "\n\n",mean_accuracy_lr)

Visualize the performance comparison for the selected hyperparameter

In [ ]:
# Set an index to select a hyperparmeter
# 0: activation function // 1: number of hidden layers // 2: learning rate
idx = 1

# Automatically define variables based on the selected index
hyper_param = ['actfn', 'layer', 'lr']
hyper_param_name = ['Activation Function', 'Num of hidden layer', 'Learning Rate']
selected = hyper_param[idx]
selected_name = hyper_param_name[idx]
exec('Result = mean_accuracy_' + hyper_param[idx])

xLabel = Result.iloc[:, 0]
x_pos = np.arange(Result.shape[0])
y_val = Result['mean']
y_err = Result['std']

# Draw a bar chart to compare the model performance (diagnostic accuracy) for each hyperparameter
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(5, 4))

# Create a bar plot with error bars
ax.bar(x_pos, y_val, yerr=y_err, align='center', alpha=0.5, ecolor='black', capsize=10,
       color=['tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:purple'])
ax.set_ylabel('Accuracy (mean)', fontsize=12)
ax.set_title(f"Performance comparsion by '{selected_name}'\n", fontsize=14)
ax.set_xticks(x_pos)
ax.set_xticklabels(xLabel, fontsize=12)
ax.yaxis.grid()
ax.set_ylim([0.6, 1.0])

plt.tight_layout()
plt.show()

Select the best model among 8 cases

In [ ]:
# Sort the Accuracy_df by 'Accuracy' column in descending order
Accuracy_df_sorted = Accuracy_df.sort_values(by='Accuracy', ascending=False).reset_index(drop=True)

# Retrieve activation function, hidden layers, and learning rate values from the first row of 'Accuracy_df_sorted'
best_actfn = Accuracy_df_sorted.iloc[0, 0]
best_layer = int(Accuracy_df_sorted.iloc[0, 1])
best_lr = Accuracy_df_sorted.iloc[0, 2]
best_accuracy = Accuracy_df_sorted.iloc[0, 3]

# Output the best case
print("[Best case]\nActivation Function: " + best_actfn +
      "\nHidden Layers: %d\nLearning Rate: %.4f\n\nAccuracy: %.2f" % (best_layer, best_lr, best_accuracy))

In [ ]:
# Load the best ANN model using the retrieved hyperparameters
best_ann_model_name = f'ANN_{best_actfn}_L{best_layer}_LR{best_lr:.4f}.h5'
best_ann_model = keras.models.load_model('YOUR_PATH/GridSearch_ANN/' + best_ann_model_name)

In [ ]:
# Print layers in the model
best_ann_model.summary()

In [ ]:
# Plot model architecture
keras.utils.plot_model(best_ann_model, dpi=80, show_shapes=True) # dpi: image resolution in dots per inch

[Confusion matrix] for the best ANN model

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix

# Predict the output (Robotic spot-welding condition) for the test data
Predicted = best_ann_model.predict(TestData)

# Convert TestLabel and Predicted into vectors to calculate the confusion matrix and evaluation metrics
TestLabel_rev = np.argmax(TestLabel, axis=1)
Predicted_rev = np.argmax(Predicted, axis=1)

# Calculate the confusion matrix
cm = confusion_matrix(TestLabel_rev, Predicted_rev)

In [ ]:
# Plot the confusion matrix
plt.figure(figsize=(6, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap=plt.cm.Blues, cbar=False, square=True)
plt.xlabel("Predicted label")
plt.ylabel("True label")
plt.title("Confusion Matrix of the Best ANN Model")
plt.show()

In [ ]:
# Normalize confusion matrix to show percentages
### You need to normalize to calculate the percentages of each result
### For class 1, (19)/(19 + 1),
### for class 2, (30)/(30 + 3), ...

cm_percent =

# Plot the percentage confusion matrix
plt.figure(figsize=(6, 6))
sns.heatmap(cm_percent, annot=True, fmt='.2f', cmap=plt.cm.Blues, cbar=False, square=True)
plt.xlabel("Predicted label")
plt.ylabel("True label")
plt.title("Confusion Matrix of the Best ANN Model (Percentage)")
plt.show()